In [23]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import os
import numpy as np
import pandas as pd
import math
import statistics
from collections import defaultdict

In [24]:
processedfiles = []
adID = []
adloc = []
adproptype = []
adprice = []
proptypes = ["flat", "apartment", "penthouse", "house", "maisonette", "villa", "field"]

In [25]:
# Set up progress bar
pbar = tqdm(total=480) 
# Looping through all 480 files
for filename in os.listdir("./data"):
    # Only consider .html files
    if filename.endswith(".html"):
    # Parse HTML into BeutifulSoup object
        html = BeautifulSoup(open("data/" + filename), "html.parser")
        # Get list of all <li> tags
        lis = html.find_all("li")
        for li in lis:
            # If this <li> tag is an ad listing, it will contain the string 'Property For Sale'
            if "Property For Sale" in li.text:
                # Convert all ad text to lowercase for consistency
                adtext = li.text.lower()

                # Extract 'name' attribute of each <li> tag. This will occasionally fail since
                # there exist other <li> tags without the 'name' attribute. In that case, 
                # I ignore the <li> tag and search for the next one. 
                # The 'name' attribute identifies each ad listing. Fortunately, if an 
                # previously parsed <li> tag reoccurs, the 'name' attribute can be used to 
                # recognize such repetition. This makes it easy to filter out overlapping data
                # later by calling drop_duplicates().
                try:
                    adID.append(li['name'])
                except:
                    continue

                # Search for municipality. In the <li> tag's raw text, 
                # these town names occur in a new line. Therefore, I 
                # use regex to match \n first. Any following characters
                # except \n are matched. Positive lookahead asserts that 
                # the characters are followed by a fullstop. First capturing
                # groups longer than 30 characters will be ignored
                loc = re.search("\n(.+?(?=\.))\.", adtext)
                if loc is None or len(loc.group(1)) > 30:
                    adloc.append("na")
                else:
                    adloc.append(loc.group(1))

                # Search for property types matching a list of 8 predefined
                # strings. If no matches are found in the ad listing, then 
                # nan is appended to the adproptype list. It is very important
                # to always append something to the list (matched string or nan)
                # otherwise the elements in the 4 lists will not correspond to
                # the same <li> tag, and drop_duplicates() will not be effective.
                match = False
                for string in proptypes:
                    if string in adtext:
                        adproptype.append(string)
                        match = True
                        break    
                if match is False:
                    adproptype.append(np.nan)

                # Search for property price. The price consistently occurs
                # after the property description sentence(s). Therefore, I 
                # exploit this by matching any characters after a fullstop
                # which are followed by any number of digits, a comma, any
                # number of digits, and finally another fullstop.
                pri = re.search("\.(.*\d+\,\d+)\.", adtext)
                if pri is None:
                    adprice.append(np.nan)
                elif pri.group(1) is None:
                    adprice.append(np.nan)
                else:
                    pri = re.search("([\d+|\,]+$)", pri.group(1))
                    adprice.append(int(pri.group(1).replace(",", "")))

        # Data has been collected for this HTML file. Now I store it 
        # as a pandas dataframe in a list for all 280 processed files
        processedfiles.append(pd.DataFrame({'id': adID,
                'price': adprice,
                'location': adloc,
                'type': adproptype}))
        adID = []
        adprice = []
        adloc = []
        adproptype = []
    
    pbar.update(1)
pbar.close()

100%|██████████| 480/480 [01:28<00:00,  5.43it/s]


In [26]:
# Ensure all HTML files have been processed. 
# The list length should be 240, the number of HTML files in /data.
len(processedfiles)

240

In [27]:
# Concatenate the list of dataframes into one dataframe.
data = pd.concat(processedfiles)
data

,id,price,location,type
0,1101104,NaN,100% upmarket properties,NaN
1,1101105,500000.0,"bungalow converted, with land",NaN
2,1101106,105000.0,"gozo, marsalforn",apartment
3,1101107,220000.0,"gozo, xewkija",NaN
4,1101108,260000.0,guardamangia / pietà,flat
...,...,...,...,...
725,1169458,NaN,na,NaN
726,1169459,875000.0,buskett,NaN
727,1169460,150000.0,pietà,NaN
728,1169461,NaN,"st julians, just off spinola",apartment


In [28]:
# Get rid of any duplicate records
data = data.drop_duplicates(subset=['id'])
data

,id,price,location,type
0,1101104,NaN,100% upmarket properties,NaN
1,1101105,500000.0,"bungalow converted, with land",NaN
2,1101106,105000.0,"gozo, marsalforn",apartment
3,1101107,220000.0,"gozo, xewkija",NaN
4,1101108,260000.0,guardamangia / pietà,flat
...,...,...,...,...
725,1169458,NaN,na,NaN
726,1169459,875000.0,buskett,NaN
727,1169460,150000.0,pietà,NaN
728,1169461,NaN,"st julians, just off spinola",apartment


In [29]:
# Function for snapping strings containing location keywords to slightly more generalized 
# locations. If no keyword is matched, location is set to nan

pbar = tqdm(total=1) 
def snaptolocation(df):
    templist = []    
    for i in range(len(df.location)):    

        cmp = str(df.location.iloc[i]) # Convert iterable to comparable string
            
        if "attard" in cmp:
            templist.append("attard")
        elif "bahar" in cmp or "baħar" in cmp:
            templist.append("baħar iċ-ċagħaq")
        elif "baħrija" in cmp:
            templist.append("baħrija")
        elif "balzvalan" in cmp:
            templist.append("balzan")
        elif "birgu" in cmp or "vittoriosa" in cmp:
            templist.append("birgu")
        elif "birkirkara" in cmp:
            templist.append("birkirkara")
        elif "birzebuġġia" in cmp or "birżebbuġa" in cmp or "qajjenza" in cmp:
            templist.append("birżebbuġa")
        elif "buġibba" in cmp:
            templist.append("buġibba")
        elif "cospicua" in cmp or "bormla" in cmp:
            templist.append("bormla")
        elif "gozo" in cmp:
            templist.append("gozo")
        elif "guardamangia" in cmp or "pieta" in cmp or "pietà" in cmp:
            templist.append("pietà")
        elif "għarghur" in cmp:
            templist.append("għarghur")
        elif "gżira" in cmp or "gzira" in cmp:
            templist.append("gżira")
        elif "hamrun" in cmp:
            templist.append("hamrun")
        elif "ibrvalag" in cmp or "ibraġ" in cmp:
            templist.append("ibraġ")
        elif "isla" in cmp or "senglea" in cmp:
            templist.append("isla")
        elif "lija" in cmp:
            templist.append("lija")
        elif "manikata" in cmp or "mellieha" in cmp or "mellieħa" in cmp:
            templist.append("mellieħa")
        elif "mosta" in cmp:
            templist.append("mosta")
        elif "msida" in cmp:
            templist.append("msida")
        elif "naxxar" in cmp:
            templist.append("naxxar")
        elif "qawra" in cmp:
            templist.append("qawra")
        elif "qormi" in cmp:
            templist.append("qormi")
        elif "sliema" in cmp:
            templist.append("sliema")
        elif "st julians" in cmp or "saint julians" in cmp:
            templist.append("st julians")
        elif "st lucia" in cmp or "santa lucia" in cmp:
            templist.append("st lucia")
        elif "st paul" in cmp or "saint paul" in cmp:
            templist.append("st paul's bay")
        elif "venera" in cmp:
            templist.append("st venera")
        elif "swatar" in cmp:
            templist.append("swatar")
        elif "xbievalx" in cmp:
            templist.append("ta' xbiex")
        elif "valletta" in cmp:
            templist.append("valletta")
        elif "xemxija" in cmp:
            templist.append("xemxija")
        elif "żabbar" in cmp or "zabbar" in cmp:
            templist.append("żabbar")
        elif "żebbug" in cmp:
            templist.append("żebbug")
        else:
            templist.append(np.nan)
        pbar.update(1)
    pbar.close()
    df.insert(3, "newlocs", templist)
    return df

  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
data = snaptolocation(data) # Snap locations

109132it [00:06, 15805.21it/s]       


In [31]:
# Replace old location column with new snapped locations column
del data['location']
data.rename(columns = {"newlocs": "location"}, inplace=True)

/home/quentin/anaconda3/envs/cps3235/lib/python3.7/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [32]:
data

,id,price,location,type
0,1101104,NaN,NaN,NaN
1,1101105,500000.0,NaN,NaN
2,1101106,105000.0,gozo,apartment
3,1101107,220000.0,gozo,NaN
4,1101108,260000.0,pietà,flat
...,...,...,...,...
725,1169458,NaN,NaN,NaN
726,1169459,875000.0,NaN,NaN
727,1169460,150000.0,pietà,NaN
728,1169461,NaN,st julians,apartment


In [33]:
# Calculate the percentages of values I was not able to extract
pricenancount = 0
locationnancount = 0
typenancount = 0

for price in data.price:
    if pd.isna(price):
        pricenancount += 1
        
for location in data.location:
    if pd.isna(location):
        locationnancount += 1

for _type in data.type:
    if pd.isna(_type):
        typenancount += 1
        
print("Price NaNs (%) \t\t" + str(((pricenancount/len(data))*100)))
print("Location NaNs (%) \t" + str(((locationnancount/len(data))*100)))
print("Property type NaNs (%) \t" + str(((typenancount/len(data))*100)))

Price NaNs (%) 		21.986218524355827
Location NaNs (%) 	39.3083605175384
Property type NaNs (%) 	10.012645236960745


In [34]:
# Prices with NaN values should be imputed in some way. Taking the median of all prices is
# a naive approach, as this will assign the median price regardless of property type. For 
# example, a villa will get an imputed price of 245k euros, which is a severe
# under-estimation of what a villa should cost. Therefore, I take the median price per 
# property type, and assign missing values based on the property type. One could take this a
# step further and find the median price for all property type and location combinations.
# In this cell I extract the median price for all 7 property types into the below dictionary.
mediansbytype = {} 

# Drop the NaN values which we want to impute later on. findmedian is a temporary dataframe.
findmedian = data.dropna(axis=0, how="any", subset=["price", "type"])
# Dictionary with list values
pricesbytype = defaultdict(list)

for i in range(len(findmedian)):
    # Append each price to a list in the dictionary based on property type
    pricesbytype[findmedian.type.iloc[i]].append(findmedian.price.iloc[i])

# Populate mediansbytype dictionary
for key, value in pricesbytype.items():
    # Calculate medians for each property type and store them in the dict
    mediansbytype.update({key: math.floor(statistics.median(value))})

mediansbytype

{'apartment': 189000,
 'flat': 295000,
 'villa': 775000,
 'maisonette': 225000,
 'penthouse': 260000,
 'house': 370000,
 'field': 55000}

In [35]:
# Drop listings of unknown property types
data = data.dropna(axis=0, how="any", subset=["type"])

In [36]:
# Impute all NaN prices with median price of the property type using a lambda function
data.price = data.apply(
    lambda row: mediansbytype[row['type']] if pd.isna(row['price']) else row['price'],
    axis = 1
)

/home/quentin/anaconda3/envs/cps3235/lib/python3.7/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [37]:
# Finally drop rows with NaNs in location column. This is my cleaned data.
data = data.dropna()
data

,id,price,location,type
2,1101106,105000.0,gozo,apartment
4,1101108,260000.0,pietà,flat
9,1100419,149000.0,attard,apartment
10,1100420,680000.0,attard,villa
11,1100421,278000.0,attard,apartment
...,...,...,...,...
714,1169520,260000.0,sliema,penthouse
716,1169522,580000.0,st julians,villa
718,1169524,170000.0,st julians,apartment
722,1169481,150000.0,mosta,apartment


In [38]:
# Send to pickle file and continue processing in a new notebook.
data.to_pickle("./cleaned_data.pkl")

In [39]:
data.to_csv(r'./cleaned_data.csv', index=False)

In [19]:
# # CODE FROM FIRST DATA COLLECTION ATTEMPT - NOT FINAL SOLUTION

# scrape = [] # List to store all ads (contains duplicates)
# historybuffer = []

# # Set up progress bar
# pbar = tqdm(total=480) 
# count = 0
# # Looping through all 480 files
# for filename in os.listdir("./data"):
#     # Only consider .html files
#     if filename.endswith(".html"):
#         # Parse HTML into BeutifulSoup object
#         html = BeautifulSoup(open("data/" + filename), "html.parser")
#         # Get list of all <h2> tags matching the class name
#         h2s = html.find_all("h2", class_ = "classified_date default_top_margin")
    
#         # Website layout varies through time. Some files give an empty h2s list.
#         # Therefore, I cater for both eventualities, starting with the newer layout.
#         if len(h2s) == 0:
#             # Get list of all <li> tags
#             lis = html.find_all("li")
#             for li in lis: # Append all ad text to scrape list
#                 adtext = li.text.replace("\n", "")
#                 if adtext not in historybuffer:
#                     scrape.append(adtext)
#                     historybuffer.append(adtext)
                    
                    
#         # Catering for the older website style
#         else:
#             # Get list of all <ul> tags
#             uls = html.find_all("ul", class_ = "classified_list")
#             for ul in uls: 
#                 # Get list of all <li> tags
#                 lis = ul.find_all("li")
#                 for li in lis: # Append all ad text to scrape list
#                     adtext = li.text.replace("\n", "")
#                     if adtext not in historybuffer:
#                         scrape.append(adtext)
#                         historybuffer.append(adtext)
                    
#     pbar.update(1)
# pbar.close()

 23%|██▎       | 110/480 [00:38<03:02,  2.02it/s]

KeyboardInterrupt: 